# Notebook 1: Fundamentos Teóricos del Transformer

Este notebook explica los componentes fundamentales de la arquitectura Transformer implementada para traducción automática neuronal (NMT) español-inglés.

## Contenido
1. Arquitectura Encoder-Decoder
2. Mecanismo de Atención
3. Máscaras de Atención
4. Codificaciones Posicionales
5. Estrategias de Decodificación

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from src.models.attention import ScaledDotProductAttention, create_causal_mask, create_padding_mask
from src.models.posenc import SinusoidalPositionalEncoding, RotaryPositionalEmbedding, ALiBiPositionalBias
from src.models.mhsa import MultiHeadAttention
from src.models.transformer import TransformerConfig, Transformer
from src.utils import set_seed

set_seed(42)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Arquitectura Encoder-Decoder

El Transformer utiliza una arquitectura encoder-decoder donde:

**Encoder:**
- Procesa la secuencia source (español) de forma bidireccional
- Cada token puede atender a todos los demás tokens
- Genera representaciones contextuales ricas

**Decoder:**
- Genera la secuencia target (inglés) de forma autoregresiva
- Self-attention con máscara causal (solo ve tokens pasados)
- Cross-attention al encoder para incorporar información del source

### Configuración del Modelo

In [ ]:
# Configuración del modelo
config = TransformerConfig(
    vocab_size_src=10000,
    vocab_size_tgt=10000,
    d_model=256,
    n_heads=8,
    num_encoder_layers=4,
    num_decoder_layers=4,
    dim_feedforward=1024,
    dropout=0.1,
    max_seq_len=100,
    pos_encoding_type='sinusoidal'
)

print(f"Configuración del Transformer:")
print(f"  - d_model: {config.d_model}")
print(f"  - n_heads: {config.n_heads}")
print(f"  - encoder_layers: {config.num_encoder_layers}")
print(f"  - decoder_layers: {config.num_decoder_layers}")
print(f"  - dim_feedforward: {config.dim_feedforward}")
print(f"  - pos_encoding: {config.pos_encoding_type}")

# Instanciar modelo
model = Transformer(config)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\nParámetros del modelo:")
print(f"  - Total: {total_params:,}")
print(f"  - Entrenables: {trainable_params:,}")

## 2. Mecanismo de Atención

La atención escalada de producto punto (Scaled Dot-Product Attention) es el componente fundamental:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

Donde:
- $Q$ (Query): representa "qué busco"
- $K$ (Key): representa "qué ofrezco"
- $V$ (Value): representa "qué información tengo"
- $d_k$: dimensión de las keys (para estabilizar gradientes)

### Visualización de Atención

In [ ]:
# Crear datos de ejemplo
batch_size, seq_len, d_model = 1, 8, 64
Q = torch.randn(batch_size, seq_len, d_model)
K = torch.randn(batch_size, seq_len, d_model)
V = torch.randn(batch_size, seq_len, d_model)

# Calcular atención
attention = ScaledDotProductAttention(dropout=0.0)
output, attn_weights = attention(Q, K, V)

# Visualizar matriz de atención
plt.figure(figsize=(8, 6))
sns.heatmap(attn_weights[0].detach().numpy(), 
            cmap='viridis', 
            annot=True, 
            fmt='.2f',
            cbar_kws={'label': 'Peso de atención'})
plt.xlabel('Posición Key/Value')
plt.ylabel('Posición Query')
plt.title('Matriz de Atención (sin máscara)')
plt.tight_layout()
plt.show()

# Verificar que los pesos suman 1
print(f"Suma de pesos por fila (debería ser ~1.0):")
print(attn_weights[0].sum(dim=-1).numpy())

## 3. Máscaras de Atención

Las máscaras controlan qué tokens pueden atender a cuáles:

### 3.1 Máscara Causal (Decoder)

Evita que el modelo vea tokens futuros durante el entrenamiento:
- Token en posición $i$ solo puede atender a posiciones $j \leq i$
- Implementada como matriz triangular inferior

In [ ]:
# Crear máscara causal
seq_len = 8
causal_mask = create_causal_mask(seq_len)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Máscara causal
sns.heatmap(causal_mask.squeeze().numpy(), 
            cmap='RdYlGn', 
            annot=True, 
            fmt='d',
            cbar_kws={'label': 'Permitido'},
            ax=axes[0])
axes[0].set_xlabel('Posición Key')
axes[0].set_ylabel('Posición Query')
axes[0].set_title('Máscara Causal\n(True=permitido, False=bloqueado)')

# Aplicar máscara a atención
output_masked, attn_weights_masked = attention(Q, K, V, mask=causal_mask)

sns.heatmap(attn_weights_masked[0].detach().numpy(), 
            cmap='viridis', 
            annot=True, 
            fmt='.2f',
            cbar_kws={'label': 'Peso de atención'},
            ax=axes[1])
axes[1].set_xlabel('Posición Key/Value')
axes[1].set_ylabel('Posición Query')
axes[1].set_title('Atención con Máscara Causal\n(triángulo superior = 0)')

plt.tight_layout()
plt.show()

print("Verificación: pesos en triángulo superior (futuro) deberían ser ~0")
print(f"Peso [0,7] (futuro): {attn_weights_masked[0, 0, 7].item():.6f}")
print(f"Peso [7,0] (pasado): {attn_weights_masked[0, 7, 0].item():.6f}")

### 3.2 Máscara de Padding

Bloquea tokens de padding (PAD) para que no contribuyan a la atención:

In [ ]:
# Secuencia con padding (0 = PAD)
seq_with_pad = torch.tensor([[1, 2, 3, 4, 5, 0, 0, 0]])  # 5 tokens reales, 3 PAD
pad_mask = create_padding_mask(seq_with_pad, pad_idx=0)

print(f"Secuencia: {seq_with_pad[0].tolist()}")
print(f"Máscara de padding (True=válido):")
print(pad_mask[0, 0].numpy())

# Visualizar efecto
output_padded, attn_weights_padded = attention(Q, K, V, mask=pad_mask)

plt.figure(figsize=(8, 6))
sns.heatmap(attn_weights_padded[0].detach().numpy(), 
            cmap='viridis', 
            annot=True, 
            fmt='.2f',
            cbar_kws={'label': 'Peso de atención'})
plt.xlabel('Posición Key/Value')
plt.ylabel('Posición Query')
plt.title('Atención con Máscara de Padding\n(columnas 5-7 deberían ser ~0)')
plt.axvline(x=5, color='red', linestyle='--', linewidth=2, label='Inicio padding')
plt.legend()
plt.tight_layout()
plt.show()

print(f"\nSuma de atención sobre tokens de padding (debería ser ~0):")
print(f"Suma columnas 5-7: {attn_weights_padded[0, :, 5:].sum().item():.6f}")

## 4. Codificaciones Posicionales

Las codificaciones posicionales añaden información de orden a las secuencias.

### 4.1 Sinusoidal (Original)

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

In [ ]:
# Crear codificación sinusoidal
max_len = 100
d_model = 128
sinusoidal_pe = SinusoidalPositionalEncoding(d_model, max_len)

# Extraer las codificaciones
pe_matrix = sinusoidal_pe.pe.squeeze(0).numpy()  # (max_len, d_model)

# Visualizar
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Heatmap completo
im = axes[0].imshow(pe_matrix.T, aspect='auto', cmap='RdBu', interpolation='nearest')
axes[0].set_xlabel('Posición')
axes[0].set_ylabel('Dimensión')
axes[0].set_title('Codificación Posicional Sinusoidal (Heatmap)')
plt.colorbar(im, ax=axes[0])

# Algunas dimensiones específicas
dims_to_plot = [0, 1, 2, 3, 64, 65]
for dim in dims_to_plot:
    axes[1].plot(pe_matrix[:, dim], label=f'dim {dim}', alpha=0.7)
axes[1].set_xlabel('Posición')
axes[1].set_ylabel('Valor de codificación')
axes[1].set_title('Codificación Posicional por Dimensión')
axes[1].legend(ncol=3)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Propiedades de la codificación sinusoidal:")
print(f"  - Dimensiones pares usan seno")
print(f"  - Dimensiones impares usan coseno")
print(f"  - Frecuencias más bajas para dimensiones altas")
print(f"  - Permite extrapolación a secuencias más largas")

### 4.2 RoPE (Rotary Position Embedding)

En lugar de sumar, RoPE rota las representaciones Q y K según la posición:

$$
q_m = R_m q, \quad k_n = R_n k
$$

donde $R_i$ es una matriz de rotación dependiente de la posición.

**Propiedad clave:** Preserva la norma de los vectores

In [ ]:
# Crear RoPE
head_dim = 64
rope = RotaryPositionalEmbedding(head_dim, max_seq_len=100)

# Crear queries y keys de prueba
batch, n_heads, seq_len, dim = 1, 8, 50, head_dim
q = torch.randn(batch, n_heads, seq_len, dim)
k = torch.randn(batch, n_heads, seq_len, dim)

# Aplicar RoPE
q_rope, k_rope = rope(q, k)

# Verificar preservación de norma
q_norm_before = torch.norm(q, dim=-1)
q_norm_after = torch.norm(q_rope, dim=-1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Comparar normas
axes[0].plot(q_norm_before[0, 0].numpy(), label='Antes de RoPE', marker='o', markersize=3)
axes[0].plot(q_norm_after[0, 0].numpy(), label='Después de RoPE', marker='x', markersize=3)
axes[0].set_xlabel('Posición')
axes[0].set_ylabel('Norma L2')
axes[0].set_title('Preservación de Norma en RoPE')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Diferencia absoluta
diff = torch.abs(q_norm_before - q_norm_after)[0, 0].numpy()
axes[1].plot(diff, marker='o', markersize=3, color='red')
axes[1].set_xlabel('Posición')
axes[1].set_ylabel('Diferencia absoluta de norma')
axes[1].set_title('Error en Preservación de Norma')
axes[1].axhline(y=1e-5, color='green', linestyle='--', label='Tolerancia (1e-5)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()

print(f"Máxima diferencia en norma: {diff.max():.2e}")
print(f"Test de preservación: {'PASS' if diff.max() < 1e-5 else 'FAIL'}")

### 4.3 ALiBi (Attention with Linear Biases)

ALiBi añade un sesgo lineal a las puntuaciones de atención:

$$
\text{score}_{i,j} = q_i \cdot k_j - m \cdot |i - j|
$$

donde $m$ es una pendiente específica de cada cabeza de atención.

**Ventaja:** Excelente extrapolación a secuencias más largas sin entrenamiento adicional

In [ ]:
# Crear ALiBi
n_heads = 8
alibi = ALiBiPositionalBias(n_heads, max_seq_len=100)

# Generar sesgos para diferentes longitudes
seq_lens = [10, 20, 50]
fig, axes = plt.subplots(1, len(seq_lens), figsize=(15, 4))

for idx, seq_len in enumerate(seq_lens):
    bias = alibi(seq_len)  # (1, n_heads, seq_len, seq_len)
    
    # Visualizar primera cabeza
    im = axes[idx].imshow(bias[0, 0].numpy(), cmap='RdYlBu_r', aspect='auto')
    axes[idx].set_xlabel('Posición Key')
    axes[idx].set_ylabel('Posición Query')
    axes[idx].set_title(f'ALiBi (seq_len={seq_len})\nCabeza 1')
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

# Verificar monotonicidad (debe decrecer con la distancia)
bias_50 = alibi(50)
print("\nVerificación de monotonicidad (fila 0, cabeza 0):")
print("Valores para posiciones consecutivas (deberían decrecer):")
for i in range(5):
    print(f"  Posición {i}: {bias_50[0, 0, 0, i].item():.4f}")

# Pendientes por cabeza
print("\nPendientes por cabeza (m):")
slopes = alibi._get_slopes(n_heads)
for i, slope in enumerate(slopes):
    print(f"  Cabeza {i}: {slope:.6f}")

### Comparación de Codificaciones Posicionales

In [ ]:
comparison_data = {
    'Método': ['Sinusoidal', 'RoPE', 'ALiBi'],
    'Parámetros adicionales': ['No', 'No', 'No'],
    'Extrapolación': ['Limitada', 'Buena', 'Excelente'],
    'Complejidad': ['Baja', 'Media', 'Baja'],
    'Preserva norma': ['No aplica', 'Sí', 'No aplica'],
    'Información': ['Absoluta', 'Relativa', 'Relativa']
}

import pandas as pd
df = pd.DataFrame(comparison_data)
print("\nComparación de Métodos de Codificación Posicional:\n")
print(df.to_string(index=False))

print("\n" + "="*80)
print("RECOMENDACIONES:")
print("="*80)
print("- Sinusoidal: Baseline estándar, bueno para secuencias dentro del rango de entrenamiento")
print("- RoPE: Mejor para capturar relaciones posicionales, popular en modelos modernos (LLaMA, GPT-NeoX)")
print("- ALiBi: Ideal si necesitas inferencia en secuencias más largas que las de entrenamiento")

## 5. Estrategias de Decodificación

Durante la inferencia, existen varias estrategias para generar secuencias:

### 5.1 Greedy Search

Selecciona el token más probable en cada paso:

$$
y_t = \arg\max P(y_t | y_{<t}, x)
$$

**Ventajas:** Rápido, determinista

**Desventajas:** Puede quedar atrapado en óptimos locales

In [ ]:
# Simulación de greedy search
vocab_size = 100
seq_len = 10

# Generar distribuciones de probabilidad simuladas
logits = torch.randn(seq_len, vocab_size)
probs = torch.softmax(logits, dim=-1)

# Greedy: seleccionar argmax
greedy_tokens = torch.argmax(probs, dim=-1)

print("Ejemplo de Greedy Search:")
print(f"Tokens seleccionados: {greedy_tokens.tolist()}")
print(f"\nProbabilidades de los tokens seleccionados:")
for i in range(seq_len):
    token_id = greedy_tokens[i].item()
    prob = probs[i, token_id].item()
    print(f"  Paso {i}: token {token_id}, prob={prob:.4f}")

### 5.2 Beam Search

Mantiene las $k$ mejores hipótesis en paralelo:

$$
\text{score}(y_{1:t}) = \sum_{i=1}^{t} \log P(y_i | y_{<i}, x)
$$

Con normalización por longitud:

$$
\text{score\_norm}(y_{1:t}) = \frac{\text{score}(y_{1:t})}{t^\alpha}
$$

donde $\alpha \approx 0.6$ evita preferencia por secuencias cortas.

In [ ]:
# Simulación de beam search
def simple_beam_search_demo(probs, beam_size=3, length_penalty=0.6):
    """
    Demostración simplificada de beam search.
    """
    vocab_size = probs.shape[1]
    
    # Inicializar con token SOS (supongamos id=1)
    beams = [{'tokens': [1], 'score': 0.0}]
    
    for step in range(5):  # Generar 5 tokens
        candidates = []
        
        for beam in beams:
            # Expandir con top-k tokens
            top_k_probs, top_k_ids = torch.topk(probs[step], beam_size)
            
            for prob, token_id in zip(top_k_probs, top_k_ids):
                new_beam = {
                    'tokens': beam['tokens'] + [token_id.item()],
                    'score': beam['score'] + torch.log(prob).item()
                }
                candidates.append(new_beam)
        
        # Seleccionar top-k candidatos con normalización por longitud
        candidates.sort(key=lambda x: x['score'] / (len(x['tokens']) ** length_penalty), reverse=True)
        beams = candidates[:beam_size]
    
    return beams

# Ejecutar demo
beams = simple_beam_search_demo(probs, beam_size=3)

print("Resultados de Beam Search (beam_size=3, length_penalty=0.6):\n")
for i, beam in enumerate(beams):
    score_norm = beam['score'] / (len(beam['tokens']) ** 0.6)
    print(f"Beam {i+1}:")
    print(f"  Tokens: {beam['tokens']}")
    print(f"  Score (log-prob): {beam['score']:.4f}")
    print(f"  Score normalizado: {score_norm:.4f}")
    print()

### 5.3 Top-k Sampling

Muestrea del top-k tokens más probables:

1. Filtrar top-k tokens
2. Renormalizar probabilidades
3. Muestrear de la distribución filtrada

In [ ]:
# Demostración de top-k sampling
k = 10
step_probs = probs[0]  # Primera distribución

# Top-k
top_k_probs, top_k_ids = torch.topk(step_probs, k)
top_k_probs_norm = top_k_probs / top_k_probs.sum()

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución original (top-20 para visualizar)
top_20_probs, top_20_ids = torch.topk(step_probs, 20)
axes[0].bar(range(20), top_20_probs.numpy())
axes[0].axvline(x=k-0.5, color='red', linestyle='--', linewidth=2, label=f'Top-{k} cutoff')
axes[0].set_xlabel('Índice (top-20)')
axes[0].set_ylabel('Probabilidad')
axes[0].set_title('Distribución Original (top-20 tokens)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Top-k renormalizado
axes[1].bar(range(k), top_k_probs_norm.numpy())
axes[1].set_xlabel('Índice (top-k)')
axes[1].set_ylabel('Probabilidad (renormalizada)')
axes[1].set_title(f'Top-{k} Sampling (renormalizado)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Top-{k} sampling:")
print(f"  Suma de probabilidades originales (top-{k}): {top_k_probs.sum():.4f}")
print(f"  Suma de probabilidades renormalizadas: {top_k_probs_norm.sum():.4f}")
print(f"  Token IDs en top-{k}: {top_k_ids.tolist()}")

### 5.4 Top-p (Nucleus) Sampling

Muestrea del conjunto más pequeño de tokens cuya probabilidad acumulada supera $p$:

$$
V^{(p)} = \min\{V' \subseteq V : \sum_{v \in V'} P(v) \geq p\}
$$

In [ ]:
# Demostración de top-p sampling
p = 0.9

# Ordenar probabilidades
sorted_probs, sorted_ids = torch.sort(step_probs, descending=True)
cumsum = torch.cumsum(sorted_probs, dim=0)

# Encontrar núcleo
nucleus_mask = cumsum <= p
# Asegurar al menos un token
nucleus_mask[0] = True

nucleus_probs = sorted_probs[nucleus_mask]
nucleus_ids = sorted_ids[nucleus_mask]
nucleus_size = nucleus_mask.sum().item()

# Visualizar
plt.figure(figsize=(12, 5))

plt.plot(cumsum[:50].numpy(), marker='o', markersize=3, label='Probabilidad acumulada')
plt.axhline(y=p, color='red', linestyle='--', linewidth=2, label=f'Umbral p={p}')
plt.axvline(x=nucleus_size-0.5, color='green', linestyle='--', linewidth=2, 
            label=f'Núcleo (tamaño={nucleus_size})')
plt.xlabel('Posición (ordenado por probabilidad)')
plt.ylabel('Probabilidad acumulada')
plt.title(f'Top-p (Nucleus) Sampling con p={p}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Top-p (nucleus) sampling con p={p}:")
print(f"  Tamaño del núcleo: {nucleus_size} tokens")
print(f"  Probabilidad acumulada del núcleo: {cumsum[nucleus_size-1].item():.4f}")
print(f"  Token IDs en núcleo (primeros 10): {nucleus_ids[:10].tolist()}")

# Comparación con top-k
print(f"\nComparación:")
print(f"  Top-k={k} siempre usa {k} tokens")
print(f"  Top-p={p} usa {nucleus_size} tokens (adaptativo)")

## Resumen

Este notebook ha cubierto los fundamentos teóricos del Transformer:

1. **Arquitectura Encoder-Decoder**: Procesamiento bidireccional del source y generación autoregresiva del target

2. **Atención**: Mecanismo de atención escalada con visualización de matrices de peso

3. **Máscaras**:
   - Causal: Bloquea información futura en el decoder
   - Padding: Ignora tokens de relleno

4. **Codificaciones Posicionales**:
   - Sinusoidal: Baseline con senos y cosenos
   - RoPE: Rotación que preserva normas
   - ALiBi: Sesgos lineales para mejor extrapolación

5. **Decodificación**:
   - Greedy: Rápido pero local
   - Beam Search: Mejor calidad con búsqueda paralela
   - Top-k: Muestreo con k tokens
   - Top-p: Muestreo adaptativo con núcleo dinámico

En los siguientes notebooks aplicaremos estos conceptos para entrenar y evaluar el modelo.